In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd
import shutil

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import max_norm
from sklearn import metrics
from sklearn.utils import class_weight
from natsort import natsort_keygen
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image
import matplotlib.cm as cm
np.random.seed(42)


In [ ]:
#it is assumed that all images of a given antibody is contained in a folder called '/.../Tiles' with two subfolders 'ABMR' and 'Other'
#creating the dataframe df3 containing all tile names, patient ID and corresponding group
#ABMR
filenames_ABMR = os.listdir('/.../Tiles/ABMR/')
filenames_ABMR = pd.DataFrame(filenames_ABMR, columns = ['tilesname'])
filenames_ABMR = filenames_ABMR.sort_values(by="tilesname",key=natsort_keygen()).reset_index(drop=True)
df = filenames_ABMR.tilesname.str.extract('(?P<tilesname>.{9})')
df = df.rename(columns={"tilesname": "patient"})
df = pd.concat([df, filenames_ABMR], axis=1)
df.insert(0, 'group', 'ABMR')
#Other
filenames_other = os.listdir('/.../Tiles/Other/')
filenames_other = pd.DataFrame(filenames_other, columns = ['tilesname'])
filenames_other = filenames_other.sort_values(by="tilesname",key=natsort_keygen()).reset_index(drop=True)
df2 = filenames_other.tilesname.str.extract('(?P<tilesname>.{9})')
df2 = df2.rename(columns={"tilesname": "patient"})
df2 = pd.concat([df2, filenames_other], axis=1)
df2.insert(0, 'group', 'Other')
#concatenate
df3 = pd.concat([df,df2])


In [ ]:

#Creating one iteration of a 3-fold cross-validation (train/validation split)
groups_by_patient_id_list = np.array(df3['patient'].values)
y_labels = df3["group"].values

n_splits = 3
gkf = StratifiedGroupKFold(n_splits = 3,shuffle=True, random_state=42)  #change random_state here for computing other iterations

result = []   
for train_idx, val_idx in gkf.split(df3, y_labels, groups = groups_by_patient_id_list):
    train_fold = df3.iloc[train_idx]
    val_fold = df3.iloc[val_idx]
    result.append((train_fold, val_fold))
    
train_fold_1, val_fold_1 = result[0][0],result[0][1]
train_fold_2, val_fold_2 = result[1][0],result[1][1]
train_fold_3, val_fold_3 = result[2][0],result[2][1]

train_fold_1 = train_fold_1.reset_index(drop=True)
val_fold_1 = val_fold_1.reset_index(drop=True)

train_fold_2 = train_fold_2.reset_index(drop=True)
val_fold_2 = val_fold_2.reset_index(drop=True)

train_fold_3 = train_fold_3.reset_index(drop=True)
val_fold_3 = val_fold_3.reset_index(drop=True)


In [ ]:
#List all file paths of the 3-fold cross validation respecting the train/validation split
def give_tiles_filepath(X_fold_X):    #X_fold_X: train/val(idation) and fold 1/2/3
    filepath = []
    for i in range(0,len(X_fold_X)):
      path = X_fold_X['group'].iloc[i] + '/' + X_fold_X['tilesname'].iloc[i]
      filepath.append(path)
    return filepath

filepath_train_f1 = give_tiles_filepath(train_fold_1)
filepath_val_f1 = give_tiles_filepath(val_fold_1)

filepath_train_f2 = give_tiles_filepath(train_fold_2)
filepath_val_f2 = give_tiles_filepath(val_fold_2)

filepath_train_f3 = give_tiles_filepath(train_fold_3)
filepath_val_f3 = give_tiles_filepath(val_fold_3)


In [ ]:

source_folder = '/.../Tiles/'  #adapt here if needed

def create_CV_directories(destination_folder,filepath_train,filepath_val):
    os.mkdir(destination_folder)
    os.mkdir(destination_folder + '/train')
    os.mkdir(destination_folder + '/train/ABMR')
    os.mkdir(destination_folder + '/train/Other')

    os.mkdir(destination_folder + '/validation')
    os.mkdir(destination_folder + '/validation/ABMR')
    os.mkdir(destination_folder + '/validation/Other')

    # copy/paste all selected files for the training dataset
    for file_name in filepath_train:
        # construct full file path
        source = source_folder + file_name
        destination = destination_folder + '/train/' + file_name
        # copy only files
        if os.path.isfile(source):
            shutil.copy(source, destination)
            print('copied', file_name)

    # copy/paste all selected files for the validation dataset
    for file_name in filepath_val:
        # construct full file path
        source = source_folder + file_name
        destination = destination_folder + '/validation/' + file_name
        # copy only files
        if os.path.isfile(source):
            shutil.copy(source, destination)
            print('copied', file_name)

#create 3 folders for each fold of the cross-validation respecting the train/validation split and ABMR/Other categories
create_CV_directories('/.../CV1',filepath_train_f1,filepath_val_f1)
create_CV_directories('/.../CV2',filepath_train_f2,filepath_val_f2)
create_CV_directories('/.../CV3',filepath_train_f3,filepath_val_f3)


In [ ]:
#Train model 1 for one fold of the cross-validation

#Select the fold
train_dir = '/.../CV1/train'  #adapt here for other folds
validation_dir = '/.../CV1/validation'  #adapt here for other folds

#Some hyperparameters
batch_size = 64
img_height = 512
img_width = 512

#Data augmentation for training and validation
#Training/validation with data augmentation
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=None,
    shear_range=0.05,
    zoom_range=0.05,
    channel_shift_range=40,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    samplewise_center=True,
    rescale = 2/255.,
)

#Without data augmentation
data = ImageDataGenerator(
    samplewise_center=True,
    rescale = 2/255.)

#Creating image generator for training and validation images with data augmentation
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    classes=None,
    class_mode="binary",
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    interpolation="bicubic",
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    classes=None,
    class_mode="binary",
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    interpolation="bicubic",
)

#Creating image generator for training and validation images without data augmentation (for input of model 2)
train_data = data.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    classes=None,
    class_mode="binary",
    batch_size=batch_size,
    shuffle=False,
    interpolation="bicubic",
)

validation_data = data.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    classes=None,
    class_mode="binary",
    batch_size=batch_size,
    shuffle=False,
    interpolation="bicubic",
)
#Dealing with potential imbalanced classes
class_weights = class_weight.compute_class_weight(
                class_weight='balanced',
                classes=np.unique(train_generator.classes), 
                y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

In [ ]:
#Functions to define
# Setup model for fine tuning
def setup_model(model, num_layer):
    #num_layer refers to the last layer to be freezed
    #Freeze the un-trainable layers of the model base
    for layer in model.layers[:num_layer]:
        layer.trainable = False

    for layer in model.layers[num_layer:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        metrics=keras.metrics.AUC(name='my_auc')
    )

filepath = 'XXX/Saved_models/checkpoint.hdf5' #adapt here for model saving

#define callbacks for saving best performing model on the validation set
callbacks_list = [
  keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_my_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    save_freq="epoch",
    options=None,
)
]

#Functions to define datasets for model 2 based on output of model 1
#function to create a dataframe including patient, tile names, predictions and corresponding group
def create_tablemaster(dataset, model):
    #Perform predictions (select image generator without data augmentation)
    predictions = model.predict(dataset, verbose=2)

    filenames = dataset.filenames
    filenames_array = np.array(filenames)
    filenames_array = np.expand_dims(filenames_array, axis=1)
    results_array = np.hstack((filenames_array,predictions))
    results_df = pd.DataFrame(results_array, columns = ['tilesname','predictions'])

    results_df[['group','tilesname']] = results_df.tilesname.str.split("/",expand=True,)
    tablemaster = results_df.tilesname.str.extract('(?P<tilesname>.{9})')
    tablemaster = tablemaster.rename(columns={"tilesname": "patient"})
    tablemaster = pd.concat([tablemaster, results_df], axis=1)
    tablemaster["predictions"] = pd.to_numeric(tablemaster.predictions, errors='coerce')
    tablemaster = tablemaster.sort_values(by="tilesname",key=natsort_keygen())
    return tablemaster

#function to define all variables for model 2 based on the output of model 1
def get_variables(tablemaster):
    #at tile level
    mean_patient = tablemaster.groupby('patient')[['predictions']].mean().reset_index()
    mean_patient = mean_patient.rename(columns={'predictions': 'mean'})

    median_patient = tablemaster.groupby('patient')[['predictions']].median().reset_index()
    median_patient = median_patient.rename(columns={'predictions': 'median'})

    min_patient = tablemaster.groupby('patient')[['predictions']].min().reset_index()
    min_patient = min_patient.rename(columns={'predictions': 'min'})
    
    max_patient = tablemaster.groupby('patient')[['predictions']].max().reset_index()
    max_patient = max_patient.rename(columns={'predictions': 'max'})

    std_patient = tablemaster.groupby('patient')[['predictions']].std().reset_index()
    std_patient = std_patient.rename(columns={'predictions': 'std'})
    
    quantile75_patient = tablemaster.groupby('patient')[['predictions']].quantile(q=0.75).reset_index()
    quantile75_patient = quantile75_patient.rename(columns={'predictions': 'quantile75'})
    
    quantile25_patient = tablemaster.groupby('patient')[['predictions']].quantile(q=0.25).reset_index()
    quantile25_patient = quantile25_patient.rename(columns={'predictions': 'quantile25'})

    #concatenate
    dataset = [mean_patient, median_patient, min_patient, max_patient, std_patient, quantile75_patient, quantile25_patient]
    dataset = pd.concat([df.set_index('patient') for df in dataset], axis=1, join='inner').reset_index()
    
    #Variables by average pooling
    tiles_value_list = tablemaster.groupby('patient')['predictions'].apply(list)
    data = []
    for i in range(0,len(tiles_value_list)):
    
        x = np.array(tiles_value_list[i])
        x = np.expand_dims(x, axis=(2, 0))
        avg_pool_1d = keras.layers.AveragePooling1D(pool_size=10,strides=5, padding='valid')
        y = avg_pool_1d(x)
        pool_quantile75 = np.quantile(y, 0.75)
        pool_quantile25 = np.quantile(y, 0.25)
        pool_minimum = np.amin(y)
        pool_maximum = np.amax(y)
        pool_std = np.std(y)
        data.append([pool_quantile75, pool_quantile25, pool_minimum, pool_maximum, pool_std])
    
    df_pooling = pd.DataFrame(data, columns=['pool_quantile75', 'pool_quantile25', 'pool_minimum', 'pool_maximum', 'pool_std'])

    idx = tiles_value_list.index
    patient = idx.to_frame(index=False, name='patient')

    df_pooling = pd.concat([patient, df_pooling], axis=1)

    #concatenate
    dataset = pd.merge(dataset, df_pooling, on ='patient', how ='left')
    
    #adding corresponding group
    tablemaster_nodups = tablemaster.drop_duplicates(subset=('patient'))
    dataset = pd.merge(dataset, tablemaster_nodups[['patient','group']], on ='patient', how ='left')
    return dataset
    
#define numpy arrays from the previous datasets
def get_the_arrays(dataset):
    x_columns = dataset.columns.drop('patient').drop('group')
    x = dataset[x_columns].values

    y = dataset['group'].map({'ABMR':1,"Other":0}).values # ABMR is 1 and Other is 0                                          
    y = np.expand_dims(y, axis=1)
    return x,y

In [ ]:
#Training of model 1
#loading ResNet50V2
base_model = keras.applications.ResNet50V2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(img_height, img_width,3),  #define input shape
    include_top=False)  # Do not include the ImageNet classifier at the top. 
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model2 = keras.Model(base_model.input, outputs)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=keras.metrics.AUC(name='my_auc'))

#Training for one epoch the added layers (convolutional blocks are kept frozen)
epochs=1
history2 = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list,
    class_weight=class_weights,
)

# Setup model for fine tuning
setup_model(model2, 86)   # defreeze conv4 and conv5
#Fine tuning for 10 epochs
epochs=10

history2 = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list,
    class_weight=class_weights,
)

#visualization of performance during training
history_dict = history2.history
acc_values = history_dict['my_auc']
val_acc_values = history_dict['val_my_auc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training')
plt.plot(epochs, val_acc_values, 'b', label='Validation')
plt.title('AUC during training and validation')
plt.xlabel('Number of epochs')
plt.ylabel('AUC')
plt.legend()
plt.show()
plt.close()

In [ ]:
#Training of model 2
#load model (if needed)
save_path = '/XXX' #folder where models are saved
model = load_model(os.path.join(save_path,"checkpoint.hdf5"))

#create dataframes (output from model 1)
train_tablemaster = create_tablemaster(train_data,model)
validation_tablemaster = create_tablemaster(validation_data,model)

#creating arrays (input for model 2), training
train_set = get_variables(train_tablemaster)
x_train, y_train = get_the_arrays(train_set)

#creating arrays (input for model 2), validaiton
test_set = get_variables(validation_tablemaster)
x_test, y_test = get_the_arrays(test_set)

#training and evaluation of model 2 
y_train_rf = np.ravel(y_train)

concatenated_auc = []
concatenated_Se_Y = []
concatenated_Sp_Y = []
concatenated_Se_C = []
concatenated_Sp_C = []
for i in range(0,50):  #averaging performances of 50 iterations
    #define and fit model
    rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, class_weight=class_weights)
    rfc.fit(x_train,y_train_rf)

    #auc score for each iteration
    rfc_predictions = rfc.predict_proba(x_test)
    auc_rfc = metrics.roc_auc_score(y_test, rfc_predictions[:,1])
    print(auc_rfc)
    #best Se and Sp
    fpr, tpr, thresholds = metrics.roc_curve(y_test, rfc_predictions[:,1])
    # Youden's statistic
    J = tpr - fpr
    ix = np.argmax(J)
    best_Se_Y = tpr[ix]
    best_Sp_Y = 1- fpr[ix]
    #closet topleft
    C = (1-tpr)**2 + (fpr)**2
    ix_C = np.argmin(C)
    best_Se_C = tpr[ix_C]
    best_Sp_C = 1 - fpr[ix_C]

    #averaging scores
    concatenated_auc.append(auc_rfc)
    concatenated_Se_Y.append(best_Se_Y)
    concatenated_Sp_Y.append(best_Sp_Y)
    concatenated_Se_C.append(best_Se_C)
    concatenated_Sp_C.append(best_Sp_C)

mean_auc = np.mean(concatenated_auc)
sample_std = np.std(concatenated_auc, ddof=1)

print('mean_auc:',mean_auc, 'sample_std:',sample_std)
print('mean_Se_Y:',np.mean(concatenated_Se_Y), 'sample_std:',np.std(concatenated_Se_Y, ddof=1))
print('mean_Sp_Y:',np.mean(concatenated_Sp_Y), 'sample_std:',np.std(concatenated_Sp_Y, ddof=1))
print('mean_Se_C:',np.mean(concatenated_Se_C), 'sample_std:',np.std(concatenated_Se_C, ddof=1))
print('mean_Sp_C:',np.mean(concatenated_Sp_C), 'sample_std:',np.std(concatenated_Sp_C, ddof=1))

In [ ]:
#GradCAM implementation (based on the keras documentation)
#training of Xception (data are exactly prepared as with ResNet50V2)
filepath = 'XXX/Saved_models/xceptionGradCAM.hdf5'  

callbacks_list = [
  keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_my_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    save_freq="epoch",
    options=None,
)
]

base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(img_height, img_width,3),
    include_top=False)  # Do not include the ImageNet classifier at the top. 
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model2 = keras.Model(base_model.input, outputs)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=keras.metrics.AUC(name='my_auc'))

epochs=1
history2 = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list,
    class_weight=class_weights,
)

# Setup model for fine tuning Xception
setup_model(model2, 116)   # defreeze conv4 et conv5
epochs=30

history2 = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list,
    class_weight=class_weights,
)

# Display
from IPython.display import Image
import matplotlib.cm as cm

img_height = 512
img_width = 512
img_size = (img_height, img_width)

#load model (if needed)
save_path = 'XXX/Saved_models' #put here the folder where models are saved
model = load_model(os.path.join(save_path,"xceptionGradCAM.hdf5"))

last_conv_layer_name = "block14_sepconv2_act"

classifier_layer_names = [
    "global_average_pooling2d_1",
    "dense_1",
]     #see model.summary() to match all layer names after last_conv_layer_name

# The local path to our target image
img_path = 'XXX/ABMR/TYMP_S013_18_(13274.0,10199.0).jpg'  #path to an image

display(Image(img_path))  #see the original tile image

In [ ]:
#based on the implementation from keras, https://keras.io/examples/vision/grad_cam/

def get_img_array(img_path, size):
    # `img` is a PIL image
    img = keras.preprocessing.image.load_img(img_path, target_size=size, color_mode="rgb", interpolation="bicubic")
    # `array` is a float32 Numpy array of shape (img_height, img_width, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, img_height, img_width, 3)
    array = np.expand_dims(array, axis=0)
    array /=127.5 
    array = array - array.mean()   #global centering
    return array

def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions 
        preds = classifier_model(last_conv_layer_output)
        if preds < 0.5:
          preds = 1 - preds
        top_class_channel = preds[:, tf.argmax(preds[0])]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
#Using GradCAM on one image tile

img_path = 'XXX/TYMP_S013_18_(13274.0,10199.0).jpg'

# Prepare image
img_array = get_img_array(img_path, size=img_size)

# Define model
model = model  #or load model

# Predict
preds = model.predict(img_array)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap)
plt.show()


In [ ]:
#Superimposed images
# We load the original image
img = keras.preprocessing.image.load_img(img_path)
img = keras.preprocessing.image.img_to_array(img)

# We rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# We use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# We use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# We create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

# Display Grad CAM
#display(Image(save_path))
display(superimposed_img)
display(preds)